### Fake News Classifier Using LSTM



In [13]:
!pip install nltk

In [14]:
import pandas as pd

df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [16]:
df = df.dropna()
df.shape

(18285, 5)

In [17]:
## Get Independent Features
X = df.drop('label',axis=1)

## Get Dependent Feature
y = df['label']

print(X.shape,y.shape) 

(18285, 4) (18285,)


### Text Preprocessing or Data Preprocessing

In [18]:
messages = X.copy()
messages.reset_index(inplace=True)
messages.head()

,index,id,title,author,text
0,0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...
1,1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...
2,2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ..."
3,3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...
4,4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...


In [19]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [27]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloading package brown to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown.zip.
[nltk_data]    | Downloading package brown_tei to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/brown_tei.zip.
[nltk_data]    | Downloading package cess_cat to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_cat.zip.
[nltk_data]    | Downloading package cess_esp to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cess_esp.zip.
[nltk_data]    | Downloading package chat80 to /root/nltk_data...
[nltk_data]    |   Unzipp

True

In [28]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem import WordNetLemmatizer
import re
from nltk.corpus import stopwords
ps = PorterStemmer()
wn = WordNetLemmatizer()
corpus = []

for i in range(0,len(messages)):
    ## Substitute all character except a-zA-Z
    review = re.sub('[^a-zA-Z]',' ',messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [wn.lemmatize(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review) ### It will make a complete sentence , If not written words will be splitted in each sentence
    corpus.append(review)
    
corpus

['house dem aide even see comey letter jason chaffetz tweeted',
 'flynn hillary clinton big woman campus breitbart',
 'truth might get fired',
 'civilian killed single u airstrike identified',
 'iranian woman jailed fictional unpublished story woman stoned death adultery',
 'jackie mason hollywood would love trump bombed north korea lack trans bathroom exclusive video breitbart',
 'beno hamon win french socialist party presidential nomination new york time',
 'back channel plan ukraine russia courtesy trump associate new york time',
 'obama organizing action partner soros linked indivisible disrupt trump agenda',
 'bbc comedy sketch real housewife isi cause outrage',
 'russian researcher discover secret nazi military base treasure hunter arctic photo',
 'u official see link trump russia',
 'yes paid government troll social medium blog forum website',
 'major league soccer argentine find home success new york time',
 'well fargo chief abruptly step new york time',
 'anonymous donor pay 

### Learn an Embedding 

In [29]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding

### One Hot Representation

In [30]:
# integer encode the documents
vocab_size = 5000
encoded_docs_one_hot = [one_hot(words, vocab_size) for words in corpus]
print(encoded_docs_one_hot)

### Each word will be converted to Integer (Index in Vocab Size)

[[2960, 4096, 3193, 1823, 4314, 749, 1431, 1574, 4854, 452], [2240, 4971, 1911, 108, 385, 1958, 1331], [4235, 4604, 1094, 1693], [3981, 1366, 4007, 3218, 596, 2078], [3715, 385, 3206, 3835, 1224, 4354, 385, 131, 3545, 312], [4747, 4952, 2483, 4462, 600, 1867, 3707, 642, 188, 2397, 3100, 2863, 76, 3946, 1331], [1138, 4132, 3553, 939, 4766, 321, 2683, 4249, 3072, 2165, 3006], [3717, 4978, 3320, 4861, 259, 1432, 1867, 1447, 3072, 2165, 3006], [3147, 2576, 1857, 4347, 1017, 1503, 3995, 3409, 1867, 1224], [1606, 3644, 3930, 192, 2286, 4824, 1718, 121], [3303, 2238, 2455, 3326, 4538, 2149, 2497, 148, 1184, 4659, 2266], [3218, 823, 4314, 1770, 1867, 259], [4452, 3694, 3873, 1, 4476, 3636, 1121, 2651, 479], [4814, 3947, 1260, 2924, 741, 3688, 3047, 3072, 2165, 3006], [1720, 1455, 376, 1986, 2957, 3072, 2165, 3006], [2414, 788, 3747, 4735, 4766, 2275, 2601, 4369, 4961, 2469], [4470, 4911, 4971], [2191, 3905, 4845, 4309, 1867, 3185, 10, 1331], [4529, 919, 1911, 825, 3502, 1126, 465, 4033, 4354],

### Using Pad Sequences 

In [31]:
max([len(sen.split(' ')) for sen in corpus ])

47

In [32]:
### Find Max no of words in Whole lists of sentence 
# sent_length=8
sent_length = max([len(sen.split(' ')) for sen in corpus ])
embedded_docs=pad_sequences(encoded_docs_one_hot,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 1574 4854  452]
 [   0    0    0 ...  385 1958 1331]
 [   0    0    0 ... 4604 1094 1693]
 ...
 [   0    0    0 ... 3072 2165 3006]
 [   0    0    0 ... 3052 4960 1222]
 [   0    0    0 ... 2977 4296  205]]


In [33]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 2960, 4096, 3193, 1823, 4314,  749, 1431,
       1574, 4854,  452], dtype=int32)

### Creating Model

In [34]:
### Now Each Word will be converted to 40 Dimensions -- 47 OF Sen1 -- Each integer of 47 will be converted to 40 features
embedding_vector_features=40 
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))  ### 1 LSTM layer with 100 Neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 47, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [35]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0, 2960, 4096, 3193, 1823, 4314,  749, 1431,
       1574, 4854,  452], dtype=int32)

In [36]:
import numpy as np

X_final = np.array(embedded_docs)
y_final = np.array(y)

X_final.shape,y_final.shape

((18285, 47), (18285,))

#### Train Test Split

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((12250, 47), (12250,), (6035, 47), (6035,))

### Model Training

In [40]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 14s 71ms/step - loss: 0.3617 - accuracy: 0.8218 - val_loss: 0.2021 - val_accuracy: 0.9163
Epoch 2/10
192/192 [==============================] - 13s 68ms/step - loss: 0.1353 - accuracy: 0.9480 - val_loss: 0.2147 - val_accuracy: 0.9188
Epoch 3/10
192/192 [==============================] - 13s 68ms/step - loss: 0.0874 - accuracy: 0.9685 - val_loss: 0.2253 - val_accuracy: 0.9183
Epoch 4/10
192/192 [==============================] - 13s 69ms/step - loss: 0.0537 - accuracy: 0.9816 - val_loss: 0.2405 - val_accuracy: 0.9178
Epoch 5/10
192/192 [==============================] - 13s 69ms/step - loss: 0.0331 - accuracy: 0.9900 - val_loss: 0.3454 - val_accuracy: 0.8968
Epoch 6/10
192/192 [==============================] - 14s 70ms/step - loss: 0.0191 - accuracy: 0.9940 - val_loss: 0.3726 - val_accuracy: 0.9152
Epoch 7/10
192/192 [==============================] - 13s 70ms/step - loss: 0.0094 - accuracy: 0.9973 - val_loss: 0.4804 - val_accuracy:

### Performance Metrics 

In [42]:
y_pred=model.predict_classes(X_test)

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9095277547638774
[[3147  272]
 [ 274 2342]]
              precision    recall  f1-score   support

           0       0.92      0.92      0.92      3419
           1       0.90      0.90      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035



### Adding Dropout 

In [44]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(vocab_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 47, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 47, 40)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [45]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((12250, 47), (12250,), (6035, 47), (6035,))

In [46]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=50,batch_size=64)

Epoch 1/50
192/192 [==============================] - 14s 74ms/step - loss: 0.3455 - accuracy: 0.8369 - val_loss: 0.2004 - val_accuracy: 0.9181
Epoch 2/50
192/192 [==============================] - 14s 71ms/step - loss: 0.1483 - accuracy: 0.9409 - val_loss: 0.2027 - val_accuracy: 0.9180
Epoch 3/50
192/192 [==============================] - 13s 70ms/step - loss: 0.1034 - accuracy: 0.9625 - val_loss: 0.2313 - val_accuracy: 0.9211
Epoch 4/50
192/192 [==============================] - 14s 70ms/step - loss: 0.0771 - accuracy: 0.9717 - val_loss: 0.2803 - val_accuracy: 0.9188
Epoch 5/50
192/192 [==============================] - 14s 70ms/step - loss: 0.0659 - accuracy: 0.9774 - val_loss: 0.2745 - val_accuracy: 0.9140
Epoch 6/50
192/192 [==============================] - 14s 71ms/step - loss: 0.0450 - accuracy: 0.9841 - val_loss: 0.3181 - val_accuracy: 0.9092
Epoch 7/50
192/192 [==============================] - 14s 71ms/step - loss: 0.0370 - accuracy: 0.9876 - val_loss: 0.3511 - val_accuracy:

In [47]:
y_pred=model.predict_classes(X_test)

from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.9030654515327258
[[3127  292]
 [ 293 2323]]
              precision    recall  f1-score   support

           0       0.91      0.91      0.91      3419
           1       0.89      0.89      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035

